In [2]:
import pandas as pd
import numpy as np



In [ ]:
#Create Vci from cluster

def liste_liste_vc(liste_cluster):

    liste_cluster

    liste_liste_vc=[]
    for cluster in liste_cluster:

        #nb cluster in this iteration
        nb_cluster= cluster["Cluster"].unique().max()

        #list
        liste_vc= []

        for i in range(nb_cluster):
            liste_vc.append((cluster["Cluster"]==i)*1)

        liste_liste_vc.append(liste_vc)
    
    return(liste_liste_vc)
    
    

In [ ]:
def K1(liste_vc1,liste_vc2):
    
    
    n=len(liste_vc1)
    m=len(liste_vc2)
    
    K1 = np.zeros((n,m))
    
    for i in range(n):
        for j in range(m):
            K1[i,j]=liste_vc1[i]@liste_vc2[j]/ (liste_vc1[i]@liste_vc1[i])
            
    return(K1)
    
    
def K2(liste_vc1,liste_vc2):
    
    
    n=len(liste_vc1)
    m=len(liste_vc2)
    
    K2 = np.zeros((n,m))
    
    for i in range(n):
        for j in range(m):
            K2[i,j]=liste_vc1[i]@liste_vc2[j]/ (liste_vc2[i]@liste_vc2[i])
            
    return(K2)    
    

In [ ]:
def birth(K2):
    return list(np.where(K2.sum(0) == 0)[0])

In [ ]:
def death(K1):
    return list(np.where(K1.sum(1) == 0)[0])

In [ ]:
def merge_growth(K1, theta):
    
    #step 1
    K1_prime= (K1<=theta)*K1
    
    
    #step 2
    maximum= np.amax(K1_prime,axis=1)
    
    for line in range(K1_prime.shape()[0]):
        K1_prime[i,:]=(K1_prime[i,:]==maximum[line])*maximum[line]
    
    
    #step 3
    

In [13]:
liste=[True,1]

In [14]:
liste

[True, 1]

In [82]:
matrix=[[0,0,3],[5,0,0]]

In [57]:
np.amax(matrix,axis=1)[0]

3

In [69]:
matrix = np.zeros((2,3))

In [45]:
np.argmax(matrix,axis=1)

array([0, 1])

In [54]:
matrix.shape

AttributeError: 'list' object has no attribute 'shape'

In [83]:
pd.notnull(matrix)

array([[ True,  True,  True],
       [ True,  True,  True]])

In [71]:
matrix

array([[0., 0., 0.],
       [0., 0., 0.]])